In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

In [2]:
import imagedataset as imgdat
import fishervector as fv

In [3]:
#Get Annotations
annotations = imgdat.GetAnnotations()

In [4]:
#Try to load all descriptors previously generated if not generate one
try:
    all_descriptors = imgdat.LoadAllDescriptors()
except:
    all_descriptors = imgdat.GenerateAllDescriptors(annotations)

Loading all sift descriptors in dataset from file...
All descriptors loaded ! 


In [5]:
#Use standardscaler for all SIFT descriptors before process them with any representation algorithm.
sc_dscs = StandardScaler()
all_descriptors = sc_dscs.fit_transform(all_descriptors)

In [6]:
#Clustering descriptors using KMeans in 120 classes 
try:
    gmm_fv = fv.LoadKmeans()
except:
    n_clusters = 10
    gmm_fv = fv.GenerateGmm(all_descriptors, n_clusters)

Training GMM of size 10
Initialization 0
  Iteration 0	 time lapse 20.68181s	 ll change inf
  Iteration 10	 time lapse 13.07780s	 ll change 0.78599
  Iteration 20	 time lapse 12.31716s	 ll change 0.26030
  Iteration 30	 time lapse 10.94147s	 ll change 0.23296
  Iteration 40	 time lapse 11.43253s	 ll change 0.09523
  Iteration 50	 time lapse 11.12592s	 ll change 0.05454
  Iteration 60	 time lapse 12.03503s	 ll change 0.02875
  Iteration 70	 time lapse 11.20593s	 ll change 0.00625
  Iteration 80	 time lapse 11.08394s	 ll change 0.04739
  Iteration 90	 time lapse 10.32381s	 ll change 0.00675
Initialization converged: False	 time lapse 133.48744s	 ll -107.46818
GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
        means_init=None, n_components=10, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=2,
        verbose_interval=10, warm_start=False, weights_init=None)
Saving gmm at: means.gmm.npy, covs.gmm.npy, weights.

C:\Anaconda2\envs\tensorflowproject\lib\site-packages\sklearn\mixture\base.py:237: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


In [7]:
#Image representation
try:
    X_fv = np.load("fv_X.npy")
    y_fv = np.load("fv_y.npy")
except:
    X_fv, y_fv = fv.FisherFeatures(annotations, gmm_fv, sc_dscs)
    np.save("fv_X", X_fv)
    np.save("fv_y", y_fv)

In [8]:
#Classification 
#Dataset splitting into the training set and test set
X_train_fv, X_test_fv, y_train_fv, y_test_fv = train_test_split(X_fv, y_fv, test_size = 0.2, random_state = 0)

In [9]:
#Scaling representation features
sc_repr_fv = StandardScaler()
X_train_fv = sc_repr_fv.fit_transform(X_train_fv)
X_test_fv = sc_repr_fv.transform(X_test_fv)

In [11]:
#Using k-nearest neighbors Classifier
clf_kn = KNeighborsClassifier(n_neighbors=3)
clf_kn.fit(X_train_fv, y_train_fv)
print('==========')
print('train score:' + str(clf_kn.score(X_train_fv,y_train_fv)))
print('test score:' + str(clf_kn.score(X_test_fv,y_test_fv)))
y_pred = clf_kn.predict(X_test_fv)
y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_test_fv, y_pred)
print(cm)

train score:0.931297709924
test score:0.787878787879
[[19 19]
 [ 2 59]]
